<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#구조적-스트리밍(Structured-Streaming)" data-toc-modified-id="구조적-스트리밍(Structured-Streaming)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>구조적 스트리밍(Structured Streaming)</a></span></li></ul></div>

# 구조적 스트리밍(Structured Streaming)

- 스트림 처리용 고수준 API
- 구조적 API(Dataset, DataFrame, SQL)로 개발된 배치 모드의 연산을 스트리밍 방식으로 실행 가능

In [3]:
staticDataFrame = spark\
                  .read.format('csv')\
                  .option('header', 'true')\
                  .option('inferSchema', 'true')\
                  .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/retail-data/by-day/*.csv')

In [4]:
# SQL 사용하기 위해 임시 테이블 등록
staticDataFrame.createOrReplaceTempView('retail_data')

# 로드한 csv 데이터들의 스키마를 객체로 할당
staticSchema = staticDataFrame.schema

In [7]:
print(staticSchema, '\n', type(staticSchema))

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,StringType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true))) 
 <class 'pyspark.sql.types.StructType'>


- 시계열 데이터를 다룰 때 타임스태프를 자주 다루기 떄문에 **윈도우 함수**를 자주 사용
- *총 구매비용 칼럼을 추가하고 고객별로 가장 많이 소비한 날*

In [8]:
staticDataFrame.show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



In [13]:
# DataFrame 구문
from pyspark.sql.functions import window, col, desc

# 그룹핑 기준을 2개: 고객별, 날짜(1일) 별
staticDataFrame\
.selectExpr("CustomerID", "(UnitPrice * Quantity) AS total_cost", "InvoiceDate")\
.groupBy(col("CustomerID"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.sort(desc("sum(total_cost)"))\
.limit(5)\
.show()

+----------+--------------------+------------------+
|CustomerID|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|[2011-09-20 09:00...|          71601.44|
|      null|[2011-11-14 09:00...|          55316.08|
|      null|[2011-11-07 09:00...|          42939.17|
|      null|[2011-03-29 09:00...| 33521.39999999998|
|      null|[2011-12-08 09:00...|31975.590000000007|
+----------+--------------------+------------------+



In [14]:
# Shuffle 파티션 수정하기
spark.conf.set('spark.sql.shuffle.partitions', '5')

- 스트리밍 코드로 변환하기
- ``maxFilesPerTrigger``로 한 번에 읽을 파일 개수 설정 가능

In [15]:
streamingDataFrame = spark\
                     .readStream\
                     .schema(staticSchema)\
                     .option('maxFilesPerTrigger', 1)\
                     .format('csv')\
                     .option('header', 'true')\
                     .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/retail-data/by-day/*.csv')

In [16]:
# 스트리밍 유형 데이터프레임인지 확인
streamingDataFrame.isStreaming

True

- 스트리밍 코드의 ``Transformations``

In [17]:
# 고객 별로 하루 당 총 판매금액 계산하는 로직
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr("CustomerID", "(UnitPrice * Quantity) AS total_cost", "InvoiceDate")\
.groupBy(col("CustomerID"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

- 코드실행위해 스트리밍 코드를 ``Action`` 하기
- 단, 스티리밍 액션은 ``count``와 같은 정적 액션과는 다름
- **트리거**가 실행된 다음 데이터를 갱신하게 될 인메모리 테이블에 데이터를 저장함
- 여기서는 파일마다 **트리거**가 실행됨


In [20]:
"""
Args:
    - memory : 인메모리 테이블에 데이터를 저장
    - customer_purchases : 인메모리에 저장될 테이블 이름
    - complete : 코드 수행 결과 모든 것을 테이블에 저장
"""
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

In [24]:
# 위에서 인메모리에 저장한 테이블에 데이터가 어떻게 기록되었는지 확인
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY 'sum(total_cost)' DESC""")\
.show(5)

+----------+--------------------+------------------+
|CustomerID|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   18075.0|[2011-06-28 09:00...|            282.23|
|   14911.0|[2011-01-31 09:00...|            797.77|
|   12921.0|[2011-03-30 09:00...|-87.30000000000001|
|   17652.0|[2011-03-03 09:00...|             222.3|
|   14506.0|[2011-11-22 09:00...|496.91999999999996|
+----------+--------------------+------------------+
only showing top 5 rows



In [26]:
# console에 출력시키기
purchaseByCustomerPerHour.writeStream\
.format("console")\
.queryName("customer_purchases_2")\
.outputMode("complete")\
.start()